In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

In [2]:
import lightgbm as lgb

In [3]:
from sklearn.model_selection import train_test_split

In [4]:
from sklearn.metrics import confusion_matrix
from sklearn.metrics import roc_auc_score

In [5]:
%matplotlib inline
%load_ext autoreload
%autoreload 2

Using loader for data

In [6]:
import sys
sys.path.append("..")

In [7]:
from helpers.picklers import *
from helpers.data_utils import *

In [8]:
#set seed
np.random.seed(42)

Data path

In [9]:
data_path = "../data/descriptor/DescriptorData.csv"

Prepare training data for our models

In [10]:
# Loading descriptor files
df = pd.read_csv(data_path)

In [11]:
# Saving in the object
X = df.loc[:, [str(i) for i in range(0, 14)]].to_numpy()
y = df.loc[:, "band_gap"].to_numpy() > 0

In [12]:
X.shape, y.shape

((18962, 14), (18962,))

In [13]:
#Split into train and test set, with test fraction 20%, startify to have equal populations in train and test
train_X, val_X, train_y, val_y = train_test_split(X, y, random_state = 42, test_size=0.2, stratify=y) 
val_X, hold_X, val_y, hold_y = train_test_split(val_X, val_y, random_state = 42, test_size=0.5, stratify=val_y) #holdout set not used for training

In [14]:
d_train = lgb.Dataset(train_X, train_y) #create lgb datasets for faster processing
d_val = lgb.Dataset(val_X, val_y)

In [15]:
def trainMod(params, d_train, itera, d_test):
    clf = lgb.train(params, d_train, num_boost_round=itera, valid_sets=[d_train, d_test])
    return clf

In [16]:
def testPred(clf, val_X, val_y):
    prob = clf.predict(val_X)
    y_pred = prob>=0.5
    print("Accuracy is:", np.mean(val_y==y_pred))
    print(confusion_matrix(val_y, y_pred))
    return y_pred, prob

In [17]:
def pred(clf, test_X):
    IDs = test_X["id"]
    test_X = test_X.drop(columns="id")
    pred = clf.predict(test_X)
    return (IDs, pred)

In [18]:
# training parameters, saved in a dict
itera=4000

In [19]:
params = {
    'learning_rate' : 0.005, #learning rate
    'data_random_seed' : 42,
    'boosting_type' : 'gbdt', #gradient boosted trees method
    'objective' : 'binary',
    'metric' : ['binary_error', 'auc'], #error metric, error function to minimize
    'feature_fraction_seed' : 42,
    'num_leaves': 100, #number of leaves in each tree, decreases bias but may overfit
    'min_data' : 50, #minimum data in each leaf, reduces overfitting
    'max_depth' : 10, #maximum depth of each tree, -1 means no limit
    'num_threads' : 4, #number of threads to use for training
    'verbosity' : 1,
    'bagging_fraction' : 0.85, # randomly bag or subsample training data. Reduces overfitting
    'feature_fraction': 0.75,  # Use subset of features for trees
    'bagging_freq' : 1, # perform bagging every Kth iteration, disabled if 0.
    'bagging_seed' : 42
}

In [20]:
#CV, 5 folds
metrics = lgb.cv(params, d_train, num_boost_round=itera, verbose_eval=100)
optimal_rounds = np.argmin(metrics["binary_error-mean"])
best_cv_score = min(metrics["binary_error-mean"])
print("\nOptimal Round: {}\nOptimal Score: {} + {}".format(
    optimal_rounds, best_cv_score, metrics["binary_error-stdv"][optimal_rounds]))

[100]	cv_agg's binary_error: 0.200145 + 0.00309035	cv_agg's auc: 0.938489 + 0.0019968
[200]	cv_agg's binary_error: 0.132243 + 0.00157263	cv_agg's auc: 0.941046 + 0.0019212
[300]	cv_agg's binary_error: 0.119125 + 0.00276082	cv_agg's auc: 0.943319 + 0.00190719
[400]	cv_agg's binary_error: 0.112268 + 0.00348042	cv_agg's auc: 0.94532 + 0.00203392
[500]	cv_agg's binary_error: 0.108906 + 0.00266049	cv_agg's auc: 0.946767 + 0.00200033
[600]	cv_agg's binary_error: 0.106138 + 0.00371336	cv_agg's auc: 0.948069 + 0.00185664
[700]	cv_agg's binary_error: 0.104094 + 0.00390837	cv_agg's auc: 0.949061 + 0.00171563
[800]	cv_agg's binary_error: 0.102644 + 0.00391525	cv_agg's auc: 0.94992 + 0.00169596
[900]	cv_agg's binary_error: 0.101193 + 0.00367679	cv_agg's auc: 0.950648 + 0.00169137
[1000]	cv_agg's binary_error: 0.100732 + 0.00509694	cv_agg's auc: 0.951279 + 0.00165221
[1100]	cv_agg's binary_error: 0.100138 + 0.00476505	cv_agg's auc: 0.951876 + 0.00160655
[1200]	cv_agg's binary_error: 0.0991496 + 0.0

In [21]:
#train model, using cv hyper parameters
model=trainMod(params, d_train, optimal_rounds, d_val)

[1]	training's binary_error: 0.24616	training's auc: 0.926094	valid_1's binary_error: 0.246308	valid_1's auc: 0.906609
[2]	training's binary_error: 0.24616	training's auc: 0.943048	valid_1's binary_error: 0.246308	valid_1's auc: 0.921707
[3]	training's binary_error: 0.24616	training's auc: 0.94679	valid_1's binary_error: 0.246308	valid_1's auc: 0.929662
[4]	training's binary_error: 0.24616	training's auc: 0.948074	valid_1's binary_error: 0.246308	valid_1's auc: 0.93116
[5]	training's binary_error: 0.24616	training's auc: 0.948815	valid_1's binary_error: 0.246308	valid_1's auc: 0.931189
[6]	training's binary_error: 0.24616	training's auc: 0.949168	valid_1's binary_error: 0.246308	valid_1's auc: 0.932838
[7]	training's binary_error: 0.24616	training's auc: 0.95015	valid_1's binary_error: 0.246308	valid_1's auc: 0.935599
[8]	training's binary_error: 0.24616	training's auc: 0.950081	valid_1's binary_error: 0.246308	valid_1's auc: 0.936083
[9]	training's binary_error: 0.24616	training's auc

[75]	training's binary_error: 0.24616	training's auc: 0.955618	valid_1's binary_error: 0.246308	valid_1's auc: 0.939688
[76]	training's binary_error: 0.24616	training's auc: 0.955644	valid_1's binary_error: 0.246308	valid_1's auc: 0.939661
[77]	training's binary_error: 0.24616	training's auc: 0.955664	valid_1's binary_error: 0.246308	valid_1's auc: 0.939623
[78]	training's binary_error: 0.24616	training's auc: 0.955805	valid_1's binary_error: 0.246308	valid_1's auc: 0.939705
[79]	training's binary_error: 0.24616	training's auc: 0.955808	valid_1's binary_error: 0.246308	valid_1's auc: 0.939708
[80]	training's binary_error: 0.24616	training's auc: 0.955897	valid_1's binary_error: 0.246308	valid_1's auc: 0.939714
[81]	training's binary_error: 0.24616	training's auc: 0.955921	valid_1's binary_error: 0.246308	valid_1's auc: 0.93967
[82]	training's binary_error: 0.24616	training's auc: 0.955989	valid_1's binary_error: 0.246308	valid_1's auc: 0.939744
[83]	training's binary_error: 0.233041	tr

[144]	training's binary_error: 0.140352	training's auc: 0.958956	valid_1's binary_error: 0.159283	valid_1's auc: 0.941102
[145]	training's binary_error: 0.139495	training's auc: 0.958974	valid_1's binary_error: 0.159283	valid_1's auc: 0.941035
[146]	training's binary_error: 0.138308	training's auc: 0.959009	valid_1's binary_error: 0.156646	valid_1's auc: 0.94106
[147]	training's binary_error: 0.137583	training's auc: 0.959075	valid_1's binary_error: 0.157173	valid_1's auc: 0.941069
[148]	training's binary_error: 0.13699	training's auc: 0.959144	valid_1's binary_error: 0.156646	valid_1's auc: 0.941042
[149]	training's binary_error: 0.136133	training's auc: 0.959175	valid_1's binary_error: 0.156646	valid_1's auc: 0.940999
[150]	training's binary_error: 0.134617	training's auc: 0.959208	valid_1's binary_error: 0.155591	valid_1's auc: 0.940985
[151]	training's binary_error: 0.13343	training's auc: 0.959239	valid_1's binary_error: 0.155063	valid_1's auc: 0.94103
[152]	training's binary_erro

[213]	training's binary_error: 0.105346	training's auc: 0.961792	valid_1's binary_error: 0.130274	valid_1's auc: 0.942079
[214]	training's binary_error: 0.104753	training's auc: 0.961837	valid_1's binary_error: 0.129747	valid_1's auc: 0.942094
[215]	training's binary_error: 0.104753	training's auc: 0.961881	valid_1's binary_error: 0.129219	valid_1's auc: 0.942105
[216]	training's binary_error: 0.104489	training's auc: 0.961902	valid_1's binary_error: 0.129219	valid_1's auc: 0.942099
[217]	training's binary_error: 0.104226	training's auc: 0.96194	valid_1's binary_error: 0.129747	valid_1's auc: 0.942153
[218]	training's binary_error: 0.104028	training's auc: 0.961967	valid_1's binary_error: 0.129747	valid_1's auc: 0.942172
[219]	training's binary_error: 0.103764	training's auc: 0.962011	valid_1's binary_error: 0.129219	valid_1's auc: 0.942219
[220]	training's binary_error: 0.103566	training's auc: 0.962041	valid_1's binary_error: 0.128692	valid_1's auc: 0.942156
[221]	training's binary_e

[284]	training's binary_error: 0.0924253	training's auc: 0.964459	valid_1's binary_error: 0.118143	valid_1's auc: 0.943293
[285]	training's binary_error: 0.0924253	training's auc: 0.964506	valid_1's binary_error: 0.117089	valid_1's auc: 0.943311
[286]	training's binary_error: 0.0924253	training's auc: 0.96455	valid_1's binary_error: 0.117089	valid_1's auc: 0.943313
[287]	training's binary_error: 0.0922276	training's auc: 0.964579	valid_1's binary_error: 0.116561	valid_1's auc: 0.943335
[288]	training's binary_error: 0.0919639	training's auc: 0.96462	valid_1's binary_error: 0.116561	valid_1's auc: 0.943359
[289]	training's binary_error: 0.0918979	training's auc: 0.964669	valid_1's binary_error: 0.117089	valid_1's auc: 0.943371
[290]	training's binary_error: 0.0915024	training's auc: 0.964703	valid_1's binary_error: 0.116034	valid_1's auc: 0.943398
[291]	training's binary_error: 0.0913046	training's auc: 0.964752	valid_1's binary_error: 0.116561	valid_1's auc: 0.943404
[292]	training's b

...

[3405]	training's binary_error: 0.0170084	training's auc: 0.998158	valid_1's binary_error: 0.0886076	valid_1's auc: 0.953184
[3406]	training's binary_error: 0.0170084	training's auc: 0.998162	valid_1's binary_error: 0.089135	valid_1's auc: 0.953176
[3407]	training's binary_error: 0.0170084	training's auc: 0.998164	valid_1's binary_error: 0.0886076	valid_1's auc: 0.953173
[3408]	training's binary_error: 0.0170084	training's auc: 0.998165	valid_1's binary_error: 0.0886076	valid_1's auc: 0.953181
[3409]	training's binary_error: 0.0170084	training's auc: 0.998168	valid_1's binary_error: 0.0886076	valid_1's auc: 0.953184
[3410]	training's binary_error: 0.0170084	training's auc: 0.998169	valid_1's binary_error: 0.0886076	valid_1's auc: 0.95318
[3411]	training's binary_error: 0.0170084	training's auc: 0.998171	valid_1's binary_error: 0.0886076	valid_1's auc: 0.95318
[3412]	training's binary_error: 0.0170084	training's auc: 0.998174	valid_1's binary_error: 0.0886076	valid_1's auc: 0.95317
[341

[3487]	training's binary_error: 0.0164151	training's auc: 0.998297	valid_1's binary_error: 0.0886076	valid_1's auc: 0.953098
[3488]	training's binary_error: 0.0163491	training's auc: 0.998298	valid_1's binary_error: 0.0886076	valid_1's auc: 0.953098
[3489]	training's binary_error: 0.0163491	training's auc: 0.998299	valid_1's binary_error: 0.0886076	valid_1's auc: 0.953102
[3490]	training's binary_error: 0.0163491	training's auc: 0.998301	valid_1's binary_error: 0.0886076	valid_1's auc: 0.953116
[3491]	training's binary_error: 0.0163491	training's auc: 0.998304	valid_1's binary_error: 0.0886076	valid_1's auc: 0.953102
[3492]	training's binary_error: 0.0163491	training's auc: 0.998306	valid_1's binary_error: 0.0886076	valid_1's auc: 0.953092
[3493]	training's binary_error: 0.0162832	training's auc: 0.998306	valid_1's binary_error: 0.089135	valid_1's auc: 0.95311
[3494]	training's binary_error: 0.0162832	training's auc: 0.99831	valid_1's binary_error: 0.0886076	valid_1's auc: 0.953101
[34

[3563]	training's binary_error: 0.015558	training's auc: 0.998418	valid_1's binary_error: 0.089135	valid_1's auc: 0.953098
[3564]	training's binary_error: 0.015558	training's auc: 0.99842	valid_1's binary_error: 0.089135	valid_1's auc: 0.953087
[3565]	training's binary_error: 0.0154921	training's auc: 0.998422	valid_1's binary_error: 0.089135	valid_1's auc: 0.953095
[3566]	training's binary_error: 0.0153603	training's auc: 0.998424	valid_1's binary_error: 0.0886076	valid_1's auc: 0.953078
[3567]	training's binary_error: 0.0154262	training's auc: 0.998426	valid_1's binary_error: 0.089135	valid_1's auc: 0.953093
[3568]	training's binary_error: 0.0153603	training's auc: 0.998427	valid_1's binary_error: 0.089135	valid_1's auc: 0.95309
[3569]	training's binary_error: 0.0153603	training's auc: 0.998429	valid_1's binary_error: 0.089135	valid_1's auc: 0.953083
[3570]	training's binary_error: 0.0153603	training's auc: 0.99843	valid_1's binary_error: 0.0886076	valid_1's auc: 0.953098
[3571]	trai

[3643]	training's binary_error: 0.0149647	training's auc: 0.998558	valid_1's binary_error: 0.0875527	valid_1's auc: 0.95312
[3644]	training's binary_error: 0.0149647	training's auc: 0.998558	valid_1's binary_error: 0.0875527	valid_1's auc: 0.953107
[3645]	training's binary_error: 0.0149647	training's auc: 0.998561	valid_1's binary_error: 0.0875527	valid_1's auc: 0.953096
[3646]	training's binary_error: 0.0149647	training's auc: 0.998562	valid_1's binary_error: 0.0875527	valid_1's auc: 0.953101
[3647]	training's binary_error: 0.0149647	training's auc: 0.998564	valid_1's binary_error: 0.0870253	valid_1's auc: 0.953105
[3648]	training's binary_error: 0.0149647	training's auc: 0.998566	valid_1's binary_error: 0.0870253	valid_1's auc: 0.953098
[3649]	training's binary_error: 0.0149647	training's auc: 0.998567	valid_1's binary_error: 0.0864979	valid_1's auc: 0.953108
[3650]	training's binary_error: 0.0149647	training's auc: 0.998569	valid_1's binary_error: 0.0864979	valid_1's auc: 0.953116
[

[3724]	training's binary_error: 0.0143055	training's auc: 0.998676	valid_1's binary_error: 0.0880802	valid_1's auc: 0.953084
[3725]	training's binary_error: 0.0142396	training's auc: 0.998678	valid_1's binary_error: 0.0880802	valid_1's auc: 0.953104
[3726]	training's binary_error: 0.0141736	training's auc: 0.998679	valid_1's binary_error: 0.0880802	valid_1's auc: 0.953104
[3727]	training's binary_error: 0.0141077	training's auc: 0.998681	valid_1's binary_error: 0.0880802	valid_1's auc: 0.953108
[3728]	training's binary_error: 0.0141077	training's auc: 0.998682	valid_1's binary_error: 0.0880802	valid_1's auc: 0.953104
[3729]	training's binary_error: 0.0141077	training's auc: 0.998685	valid_1's binary_error: 0.0880802	valid_1's auc: 0.953095
[3730]	training's binary_error: 0.0141736	training's auc: 0.998688	valid_1's binary_error: 0.0880802	valid_1's auc: 0.953092
[3731]	training's binary_error: 0.0141736	training's auc: 0.998688	valid_1's binary_error: 0.0880802	valid_1's auc: 0.953083


In [22]:
pred, prob = testPred(model, hold_X, hold_y) #test on holdout set not used for training in any way
auc = roc_auc_score(hold_y, prob)
print("AUC is:", auc)

Accuracy is: 0.9024775962045335
[[1355   75]
 [ 110  357]]
AUC is: 0.9546053518216259


In [24]:
dict1 = {'Pred_LGBM': prob, 'Real_LGBM': hold_y}  
df = pd.DataFrame(dict1)
df.to_csv("LGBMData.csv", index=False)